In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline



![img](https://github.com/yandexdataschool/nlp_course/raw/master/resources/banhammer.jpg)

__In this notebook__ you will build an algorithm that classifies social media comments into normal or toxic.
Like in many real-world cases, you only have a small (10^3) dataset of hand-labeled examples to work with. We'll tackle this problem using both classical nlp methods and embedding-based approach.

In [2]:
import pandas as pd
data = pd.read_csv("comments.tsv", sep='\t')

texts = data['comment_text'].values
target = data['should_ban'].values
data[50::200]

,should_ban,comment_text
50,0,"""Those who're in advantageous positions are th..."
250,1,Fartsalot56 says f**k you motherclucker!!
450,1,"Are you a fool? \n\nI am sorry, but you seem t..."
650,1,I AM NOT A VANDAL!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
850,0,Citing sources\n\nCheck out the Wikipedia:Citi...


In [3]:
from sklearn.model_selection import train_test_split
texts_train, texts_test, y_train, y_test = train_test_split(texts, target, test_size=0.5, random_state=42)


__Note:__ it is generally a good idea to split data into train/test before anything is done to them.

It guards you against possible data leakage in the preprocessing stage. For example, should you decide to select words present in obscene tweets as features, you should only count those words over the training set. Otherwise your algoritm can cheat evaluation.

### Preprocessing and tokenization

Comments contain raw text with punctuation, upper/lowercase letters and even newline symbols.

To simplify all further steps, we'll split text into space-separated tokens using one of nltk tokenizers.

In [4]:
from nltk.tokenize import TweetTokenizer
tokenizer = TweetTokenizer()
preprocess = lambda text: ' '.join(tokenizer.tokenize(text.lower()))

text = 'How to be a grown-up at work: replace "fuck you" with "Ok, great!".'
print("before:", text,)
print("after:", preprocess(text),)
# print(type(text))

before: How to be a grown-up at work: replace "fuck you" with "Ok, great!".
after: how to be a grown-up at work : replace " fuck you " with " ok , great ! " .


In [5]:
# task: preprocess each comment in train and test

texts_train = preprocess(str(texts_train)) #<YOUR CODE>
texts_test = preprocess(str(texts_test))

In [6]:
len(texts_train)

212415

In [7]:
try:
  assert texts_train[5] ==  'who cares anymore . they attack with impunity .'
  assert texts_test[89] == 'hey todds ! quick q ? why are you so gay'
  assert len(texts_test) == len(y_test)
except AssertionError:
  print("assetion error")
print(len(texts_train))

assetion error
212415


### Solving it: bag of words

![img](http://www.novuslight.com/uploads/n/BagofWords.jpg)

One traditional approach to such problem is to use bag of words features:
1. build a vocabulary of frequent words (use train data only)
2. for each training sample, count the number of times a word occurs in it (for each word in vocabulary).
3. consider this count a feature for some classifier

__Note:__ in practice, you can compute such features using sklearn. Please don't do that in the current assignment, though.
* `from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer`

In [8]:
# task: find up to k most frequent tokens in texts_train,
# sort them by number of occurences (highest first)
from collections import Counter

k = 10000

# <YOUR CODE>
split_it = texts_train.split()    
counter_obj = Counter(split_it)
bow_vocabulary = dict(counter_obj.most_common(n = k))
text, count = bow_vocabulary.keys(), bow_vocabulary.values()
bow_vocabulary = list(text)

# split_it = texts_train.split()
# counter_obj = Counter(split_it)
# bow_vocabulary = dict(counter_obj.most_common(n = k))
# bow_vocabulary = list(sorted(bow_vocabulary.items(), key=lambda item: item[1], reverse=True))
print('example features:', bow_vocabulary)


example features: ['\\', '.', ',', '"', 'you', 'the', "'", 'a', 'n', 'to', 'and', '!', 'i', 'is', 'of', '=', 'suck', 'that', 'it', 'hitler', 'heil', 'in', '(', ')', 'offfuck', '?', 'are', 'not', 'this', 'for', 'be', 'your', 'my', 'on', ':', 'with', 'as', 'do', 'have', 'nigger', 'j.delanoy', 'me', 'dick', 'or', 'was', 'article', 'if', 'know', 'at', 'page', 'but', 'what', 'all', 'an', '...', 'so', 'talk', 'by', 'wikipedia', 's', 'he', 'like', 'they', 'one', 'can', 'from', 'no', 'will', 'has', 'would', 'been', 'there', 'fggt', 'about', 'nbecause', 'just', 'which', '2006', 'any', 'ni', 'did', '-', 'who', 'time', 'when', 't', 'we', 'july', 'hist', 'diff', '்', '24', 'people', 'had', 'his', 'please', 'should', '|', 'them', 'see', 'here', 'up', 'because', 'think', "don't", 'other', 'before', 'some', 'how', ';', 'bitch', 'am', 'make', 'also', 'out', '/', 'then', 'being', 'may', 'only', 'user', 'why', 'more', 'use', 'articles', 'again', 'their', 'even', 'fuck', 'edit', 'now', 'does', 'get', 'go

In [9]:
def text_to_bow(text):
    """ convert text string to an array of token counts. Use bow_vocabulary. """
    # <YOUR CODE>
    text = preprocess(str(text))
    split_it = text.split()    
    array = list()
    counter_obj = Counter(split_it)
    bow_vocabulary = dict(counter_obj.most_common(n = k))

    text, count = bow_vocabulary.keys(), bow_vocabulary.values()
    
    array = np.array(list(count), 'float32').flatten()
    text = list(bow_vocabulary)
    return array

text = """Life changes fast. Life changes in the instant. You sit down to dinner and life as you know it ends. The question of self-pity."""
text_to_bow(text)

array([4., 3., 2., 2., 2., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1.], dtype=float32)

In [10]:
def text_to_bow(text):
    """ convert text string to an array of token counts. Use bow_vocabulary. """
    # <YOUR CODE>

    text = preprocess(str(text))
    split_it = text.split()    
    array = np.zeros(len(bow_vocabulary))

    # print(bow_vocabulary.keys)
    # text, count = bow_vocabulary.keys(), bow_vocabulary.values()
    for i in split_it:
      try:
        index = bow_vocabulary.index(i)
        array[index] += 1
      except ValueError:
        pass
      
    array = np.array(array, 'float32')
    return array

text = """Life changes fast. Life changes in the instant. You sit down to dinner and life as you know it ends. The question of self-pity."""
text_to_bow(texts_train)

array([1.771e+03, 1.516e+03, 1.389e+03, ..., 1.000e+00, 1.000e+00,
       1.000e+00], dtype=float32)

In [ ]:

# X_train_bow = np.array(list(text_to_bow(texts_train)))
# X_train_bow = X_train_bow[X_train_bow is not None]
# X_train_bow = np.stack(X_train_bow)
# # X_train_bow.reshape(1, -1)

# X_test_bow = np.array(list(text_to_bow(texts_test)))
# X_test_bow = X_train_bow[X_test_bow is not None]
# X_test_bow = np.stack(X_test_bow)
# # X_test_bow.reshape(1, -1)
X_train_bow = np.stack(list(map(text_to_bow, texts_train)))
X_test_bow = np.stack(list(map(text_to_bow, texts_test)))

In [ ]:
X_train_bow[:10]

In [ ]:
k_max = len(set(' '.join(texts_train).split()))
try:
  assert X_train_bow.shape == (len(texts_train), min(k, k_max))
  assert X_test_bow.shape == (len(texts_test), min(k, k_max))
  assert np.all(X_train_bow[5:10].sum(-1) == np.array([len(s.split()) for s in  texts_train[5:10]]))
  assert len(bow_vocabulary) <= min(k, k_max)
  assert X_train_bow[6, bow_vocabulary.index('.')] == texts_train[6].split().count('.')
except AssertionError:
  print("assetion error")

Machine learning stuff: fit, predict, evaluate. You know the drill.

In [ ]:
from sklearn.linear_model import LogisticRegression
bow_model = LogisticRegression().fit(X_train_bow, y_train)

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve

for name, X, y, model in [
    ('train', X_train_bow, y_train, bow_model),
    ('test ', X_test_bow, y_test, bow_model)
]:
    proba = model.predict_proba(X)[:, 1]
    auc = roc_auc_score(y, proba)
    plt.plot(*roc_curve(y, proba)[:2], label='%s AUC=%.4f' % (name, auc))

plt.plot([0, 1], [0, 1], '--', color='black',)
plt.legend(fontsize='large')
plt.grid()

```

```

```

```

```

```

```

```

```

```

```

```

```

```

```

```


### Solving it better: word vectors

Let's try another approach: instead of counting per-word frequencies, we shall map all words to pre-trained word vectors and average over them to get text features.

This should give us two key advantages: (1) we now have 10^2 features instead of 10^4 and (2) our model can generalize to word that are not in training dataset.

We begin with a standard approach with pre-trained word vectors. However, you may also try
* training embeddings from scratch on relevant (unlabeled) data
* multiplying word vectors by inverse word frequency in dataset (like tf-idf).
* concatenating several embeddings
    * call `gensim.downloader.info()['models'].keys()` to get a list of available models
* clusterizing words by their word-vectors and try bag of cluster_ids

__Note:__ loading pre-trained model may take a while. It's a perfect opportunity to refill your cup of tea/coffee and grab some extra cookies. Or binge-watch some tv series if you're slow on internet connection

In [ ]:
import gensim.downloader 
embeddings = gensim.downloader.load("fasttext-wiki-news-subwords-300")

# If you're low on RAM or download speed, use "glove-wiki-gigaword-100" instead. Ignore all further asserts.

In [ ]:
def vectorize_sum(comment):
    """
    implement a function that converts preprocessed comment to a sum of token vectors
    """
    embedding_dim = embeddings.wv.vectors.shape[1]
    features = np.zeros([embedding_dim], dtype='float32')
    
    # <YOUR CODE>

    for word in preprocess(comment).split():
      if word in embeddings.key_to_index:
        features += embeddings.vectors[embeddings.key_to_index[word]]

    
    return features

assert np.allclose(
    vectorize_sum("who cares anymore . they attack with impunity .")[::70],
    np.array([ 0.0108616 ,  0.0261663 ,  0.13855131, -0.18510573, -0.46380025])
)

In [ ]:
X_train_wv = np.stack([vectorize_sum(text) for text in texts_train])
X_test_wv = np.stack([vectorize_sum(text) for text in texts_test])

In [ ]:
wv_model = LogisticRegression().fit(X_train_wv, y_train)

for name, X, y, model in [
    ('bow train', X_train_bow, y_train, bow_model),
    ('bow test ', X_test_bow, y_test, bow_model),
    ('vec train', X_train_wv, y_train, wv_model),
    ('vec test ', X_test_wv, y_test, wv_model)
]:
    proba = model.predict_proba(X)[:, 1]
    auc = roc_auc_score(y, proba)
    plt.plot(*roc_curve(y, proba)[:2], label='%s AUC=%.4f' % (name, auc))

plt.plot([0, 1], [0, 1], '--', color='black',)
plt.legend(fontsize='large')
plt.grid()

assert roc_auc_score(y_test, wv_model.predict_proba(X_test_wv)[:, 1]) > 0.92, "something's wrong with your features"

If everything went right, you've just managed to reduce misclassification rate by a factor of two.
This trick is very useful when you're dealing with small datasets. However, if you have hundreds of thousands of samples, there's a whole different range of methods for that. We'll get there in the second part.